Problem I - Initial value problem
$$
y' = \frac{1}{x^2} - \frac{y}{x}, \; 1 \le x \le 2 \; \text{with} \; y(1) = 1.
$$
The exact solution is
$$
y = \frac{1 + \ln x}{x}
$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
def f(t, y):
    return 1 / t**2 - y / t

In [ ]:
def Forward_Euler(f, init, T, dt):
    t = np.arange(1, T+dt, dt)
    nt = t.shape[0]
    y = np.zeros(nt)
    
    y[0] = init

    for n in range(nt-1):
        y[n+1] = y[n] + f(t[n], y[n])*dt
        
    return t, y

In [ ]:
def RK2(f, init, T, dt):
    t = np.arange(1, T+dt, dt)
    nt = t.shape[0]
    y = np.zeros(nt)
    
    y[0] = init

    for n in range(nt-1):
        k1 = f(t[n], y[n])
        k2 = f(t[n] + 2.0 / 3.0 * dt, y[n] + 2.0 / 3.0 * dt * k1)
        y[n+1] = y[n] + (k1 + 3.0 * k2) * dt / 4.0
        
    return t, y

In [ ]:
def RK3(f, init, T, dt):
    t = np.arange(1, T+dt, dt)
    nt = t.shape[0]
    y = np.zeros(nt)
    
    y[0] = init

    for n in range(nt-1):
        k1 = f(t[n], y[n])
        k2 = f(t[n] + 0.5 * dt, y[n] + 0.5 * dt * k1)
        k3 = f(t[n+1], y[n] - dt * k1 + 2.0 * dt * k2)
        y[n+1] = y[n] + (k1 + 4.0 * k2 + k3) * dt / 6
        
    return t, y

In [ ]:
def RK4(f, init, T, dt):
    t = np.arange(1, T+dt, dt)
    nt = t.shape[0]
    y = np.zeros(nt)
    
    y[0] = init

    for n in range(nt-1):
        k1 = f(t[n], y[n])
        k2 = f(t[n] + 0.5 * dt, y[n] + 0.5 * dt * k1)
        k3 = f(t[n] + 0.5 * dt, y[n] + 0.5 * dt * k2)
        k4 = f(t[n+1], y[n] + dt * k3)
        y[n+1] = y[n] + (k1 + 2.0 * k2 + 2.0 * k3 + k4) * dt / 6.0
        
    return t, y

In [ ]:
def BDF2(f, init, T, dt):
    t = np.arange(1, T+dt, dt)
    nt = t.shape[0]
    y = np.zeros(nt)
    Iter = 100
    y[0] = init
    y[1] = y[0] +  f(t[0], y[0]) * dt 
    
    for n in range(nt-2):
        for j in range(Iter):
            y[n+2] = - y[n] / 3.0 + y[n+1] * 4.0 / 3.0 + f(t[n+2], y[n+2]) * dt * 2.0 / 3.0
        
    return t, y

In [ ]:
y_ex = lambda x: (1.0 + np.log(x)) / x

In [ ]:
h = 0.05
# %time t_num, y_num = Forward_Euler(f, 1.0, 2.0, h)
# %time t_num, y_num = RK2(f, 1.0, 2.0, h)
# %time t_num, y_num = RK3(f, 1.0, 2.0, h)
# %time t_num, y_num = RK4(f, 1.0, 2.0, h)
%time t_num, y_num = BDF2(f, 1.0, 2.0, h)

plt.figure(figsize=(8,8))
t = np.linspace(1.0, 2.0, 100)
plt.plot(t_num, y_num, 'x', t, y_ex(t))

In [ ]:
# test order
h_list = [0.2, 0.1, 0.05, 0.02, 0.01, 0.001]
error_list = []
for h in h_list:
    t_num, y_num = BDF2(f, 1.0, 2.0, h)
    y_exact = y_ex(t_num[-1])
    # print(abs( y_ex(t_num[-1]) - y_num[-1]))
    err = abs( y_ex(t_num[-1]) - y_num[-1])
    error_list.append(err)

In [ ]:
h4_list = [h**2 for h in h_list]
plt.loglog(h_list, error_list, "rx", h_list, h4_list, "-.")

Problem II - Lorenz equation
$$
\begin{equation}
    \left\{
    \begin{aligned}
         & \frac{\mathbf{d} x}{\mathbf{d} t} = \sigma (y - x),            \\
         & \frac{\mathbf{d} y}{\mathbf{d} t} = x (\rho - z) - y ,           \\
         & \frac{\mathbf{d} z}{\mathbf{d} t} = xy - \beta z.
    \end{aligned}
    \right.
\end{equation}
$$
#### with $x(0) = y(0) = z(0) = 1$.

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

mpl.rcParams['legend.fontsize'] = 10


In [ ]:
def F(sigma, rho, beta, x, y, z):
    return sigma * (y - x), x * (rho - z) - y, x * y - beta * z

In [ ]:
def lorenz_solver(F, init, T, dt):

    sigma = 10
    rho = 28
    beta = 8.0 / 3.0

    t = np.arange(0, T+dt, dt)
    nt = t.shape[0]
    x = np.zeros(nt)
    x[0] = init[0]
    y = np.zeros(nt)
    y[0] = init[1]
    z = np.zeros(nt)
    z[0] = init[-1]

    for n in range(nt-1):
        
        k1, l1, m1 = F(sigma, rho, beta, x[n], y[n], z[n])
        k2, l2, m2 = F(sigma, rho, beta, x[n] + 0.5 * dt * k1, y[n] + 0.5 * dt * l1, z[n] + 0.5 * dt * m1)
        k3, l3, m3 = F(sigma, rho, beta, x[n] + 0.5 * dt * k2, y[n] + 0.5 * dt * l2, z[n] + 0.5 * dt * m2)
        k4, l4, m4 = F(sigma, rho, beta, x[n] + dt * k3, y[n] + dt * l3, z[n] + dt * m3)   

        x[n+1] = x[n] + (k1 + 2.0 * k2 + 2.0 * k3 + k4) * dt / 6.0
        y[n+1] = y[n] + (l1 + 2.0 * l2 + 2.0 * l3 + l4) * dt / 6.0
        z[n+1] = z[n] + (m1 + 2.0 * m2 + 2.0 * m3 + m4) * dt / 6.0

    return t, x, y, z

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')

h = 0.001
t_num, x_num, y_num, z_num = lorenz_solver(F = F, init = [1.0, 1.0, 1.0], T = 20.0, dt = h)

ax.plot(x_num, y_num, z_num, label='parametric curve')
ax.legend()
plt.show()